In [2]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())

In [3]:
from qiskit_nature.algorithms import NumPyEigensolverFactory

numpy_solver = NumPyEigensolverFactory(use_default_filter_criterion=True)

In [4]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory

# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
solver = VQEUCCFactory(quantum_instance)
gsc = GroundStateEigensolver(qubit_converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")

In [5]:
from qiskit_nature.algorithms import ExcitedStatesEigensolver

numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_excited_states_calculation.solve(es_problem)

qeom_results = qeom_excited_states_calculation.solve(es_problem)

print(numpy_results)
print("\n\n")

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -0.882722150245
* Electronic excited state energy (Hartree): -0.882722150245
> Total excited state energy (Hartree): -0.162753155796
  2: 
* Electronic excited state energy (Hartree): -0.224911252831
* Electronic excited state energy (Hartree): -0.224911252831
> Total excited state energy (Hartree): 0.495057741618
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed pa

In [8]:
print(qeom_results.eigenenergies[1], qeom_results.eigenenergies[0] )
bandgap = qeom_results.computed_energies[1] - qeom_results.computed_energies[0]
bandgap

(-1.2445867793629346+0j) (-1.857275030143452+0j)


0.6126882507805174

In [12]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
print(len(second_q_op))

7


In [13]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper 
qubit_converter = QubitConverter(JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

-0.8105479805373268 * IIII
- 0.225753492224025 * ZIII
+ 0.17218393261915543 * IZII
+ 0.12091263261776641 * ZZII
- 0.225753492224025 * IIZI
+ 0.17464343068300464 * ZIZI
+ 0.1661454325638243 * IZZI
+ 0.17218393261915546 * IIIZ
+ 0.1661454325638243 * ZIIZ
+ 0.16892753870087926 * IZIZ
+ 0.12091263261776641 * IIZZ
+ 0.04523279994605788 * XXXX
+ 0.04523279994605788 * YYXX
+ 0.04523279994605788 * XXYY
+ 0.04523279994605788 * YYYY


In [14]:
from qiskit_nature.circuit.library import HartreeFock
init_state = HartreeFock(4, (1,1), qubit_converter)
init_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

In [15]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits = 4, 
                      su2_gates = ['h', 'ry'],
                      entanglement = 'full',
                      reps = 2,
                      initial_state = init_state)
ansatz.decompose().draw()

┌───┐   ┌───┐    ┌──────────┐                    ┌───┐┌──────────┐»
q_0: ┤ X ├───┤ H ├────┤ Ry(θ[0]) ├──■────■─────────■──┤ H ├┤ Ry(θ[4]) ├»
     ├───┤┌──┴───┴───┐└──────────┘┌─┴─┐  │         │  └───┘└──┬───┬───┘»
q_1: ┤ H ├┤ Ry(θ[1]) ├────────────┤ X ├──┼────■────┼────■─────┤ H ├────»
     ├───┤└──┬───┬───┘┌──────────┐└───┘┌─┴─┐┌─┴─┐  │    │     └───┘    »
q_2: ┤ X ├───┤ H ├────┤ Ry(θ[2]) ├─────┤ X ├┤ X ├──┼────┼───────■──────»
     ├───┤┌──┴───┴───┐└──────────┘     └───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐    »
q_3: ┤ H ├┤ Ry(θ[3]) ├───────────────────────────┤ X ├┤ X ├───┤ X ├────»
     └───┘└──────────┘                           └───┘└───┘   └───┘    »
«                                            ┌───┐┌──────────┐            »
«q_0: ─────────────────■────────■─────────■──┤ H ├┤ Ry(θ[8]) ├────────────»
«     ┌──────────┐   ┌─┴─┐      │         │  └───┘└──┬───┬───┘┌──────────┐»
«q_1: ┤ Ry(θ[5]) ├───┤ X ├──────┼────■────┼────■─────┤ H ├────┤ Ry(θ[9]) ├»
«     └──┬───┬───┘┌──┴───┴───┐┌─┴─┐┌─┴─┐  │    │     └───┘    └──┬───┬───┘»
«q_2: ───┤ H ├────┤ Ry(θ[6]) ├┤ X ├┤ X ├──┼────┼───────■─────────┤ H ├────»
«        ├───┤    ├──────────┤└───┘└───┘┌─┴─┐┌─┴─┐   ┌─┴─┐       ├───┤    »
«q_3: ───┤ H ├────┤ Ry(θ[7]) ├──────────┤ X ├┤ X ├───┤ X ├───────┤ H ├────»
«        └───┘    └──────────┘          └───┘└───┘   └───┘       └───┘    »
«                  
«q_0: ─────────────
«                  
«q_1: ─────────────
«     ┌───────────┐
«q_2: ┤ Ry(θ[10]) ├
«     ├───────────┤
«q_3: ┤ Ry(θ[11]) ├
«     └───────────┘

In [16]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit.algorithms.optimizers import COBYLA
backend = StatevectorSimulator()
optimizer = COBYLA()

In [17]:
import numpy as np
np.random.seed(5)
num_params = ansatz.num_parameters
initial_point = np.random.random(num_params)

In [18]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import GroundStateEigensolver
vqe = VQE(ansatz = ansatz,
         optimizer = optimizer,
         initial_point = initial_point,
         quantum_instance = backend)

vqe_ground_state_solver = GroundStateEigensolver(qubit_converter, vqe)
vqe_results = vqe_ground_state_solver.solve(es_problem)
print(vqe_results.eigenenergies[0])

(-1.8370382630399187+0j)


In [19]:
from qiskit_nature.algorithms import QEOM
qeom_excited_state_solver = QEOM(vqe_ground_state_solver, 'sd')
qeom_results = qeom_excited_state_solver.solve(es_problem)
print(qeom_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.83703826304
  - computed part:      -1.83703826304
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.117069268591
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -1.272636387067
* Electronic excited state energy (Hartree): -1.272636387067
> Total excited state energy (Hartree): -0.552667392618
  2: 
* Electronic excited state energy (Hartree): -0.899972765749
* Electronic excited state energy (Hartree): -0.899972765749
> Total excited state energy (Hartree): -0.1800037713
  3: 
* Electronic excited state energy (Hartree): -0.245142417372
* Electronic excited state energy (Hartree): -0.245142417372
> Total excited state energy (Hartree): 0.474826577077
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  *

In [20]:
bandgap = qeom_results.computed_energies[1] - qeom_results.computed_energies[0]
bandgap

0.5644018759726352

In [41]:
from qiskit.opflow import CircuitStateFn, StateFn

from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki
from qiskit.circuit.parametervector import ParameterVector

H_op = qubit_op
ansatz = ansatz
ansatz.parameters

# problem solved
# ansatz has a parameterView
# but to pull out parameter vector we should get a parameter
# and then call it's vector parent which will be parameterVector
ry = ansatz.parameters[0].vector
ry

ParameterVector(name=θ, length=12)

In [42]:
H_meas = StateFn(H_op, is_measurement=True)
circuit_ansatz = CircuitStateFn(ansatz)
H_and_ansatz = H_meas @ circuit_ansatz

sampler = CircuitSampler(backend = backend)
params = np.random.random(num_params)
binded = H_and_ansatz.bind_parameters({ry:params})
sampled = sampler.convert(binded)
print(sampled.eval())

(-1.3361194913805563+0j)


In [43]:
from scipy.optimize import minimize
from qiskit import *
def cost(params):
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    sampler = CircuitSampler(backend = backend)
    binded = H_and_ansatz.bind_parameters({ry:params})
    sampled = sampler.convert(binded)
    return sampled.eval().real
params = np.random.random(num_params)
vqe_result = minimize(cost, params, method="Powell", tol=1e-3)

print('The vqe ground state energy using is: {}'.format(vqe_results.eigenenergies[0]))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))


The vqe ground state energy using is: (-1.8370382630399187+0j)
The estimated ground state energy from VQE algorithm is: -1.836967991202985


In [44]:
optimal_params = vqe_result.x
optimal_params

array([-0.85224443, -0.97393376, -1.43096557,  1.26938671,  0.00241006,
       -0.04317803, -0.01186481,  0.29484183,  0.01267098,  0.71965133,
        0.62607604,  0.09501496])

In [45]:
opt_circ = ansatz.bind_parameters({ry:optimal_params})
zero_proj = (1 / 2**4 * (I + Z) ^ (I + Z) ^ (I + Z) ^ (I + Z))

zero_proj_meas = StateFn(zero_proj, is_measurement = True)
ansatz_and_opt_circ = ansatz.compose(opt_circ.inverse())
#Penalty_ansatz = CircuitStateFn(ansatz_and_opt_circ)
Penalty_ansatz = CircuitStateFn(ansatz + opt_circ.inverse())
Penalty_and_ansatz = zero_proj_meas @ Penalty_ansatz

H_tot = H_and_ansatz + Penalty_and_ansatz

def cost(params):
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    sampler = CircuitSampler(backend = backend)
    binded = H_tot.bind_parameters({ry:params})
    sampled = sampler.convert(binded)
    return sampled.eval().real
params = np.random.random(num_params)
vqd_result = minimize(cost, params, method="Powell", tol=1e-3)



In [46]:
print('The qeom excited state energy using is: {}'.format(qeom_results.eigenenergies[1]))
print('The estimated excited state energy from vqd algorithm is: {}'.format(vqd_result.fun))


The qeom excited state energy using is: (-1.2726363870672834+0j)
The estimated excited state energy from vqd algorithm is: -1.2518727658041273


In [47]:
print(qeom_results.eigenenergies[2], qeom_results.eigenenergies[1] )
bandgap2 = qeom_results.computed_energies[2] - qeom_results.computed_energies[1]
bandgap2

(-0.8999727657488661+0j) (-1.2726363870672834+0j)


0.37266362131841735

In [48]:
optimal_params = vqd_result.x
optimal_params

array([-0.29204294,  1.99982226, -0.91547932,  1.71286972,  0.78134214,
        0.77699373, -0.34081794, -0.11978442,  0.3868845 ,  1.23840849,
        0.51798508,  0.74359282])

In [51]:
opt_circ2 = ansatz.bind_parameters({ry:optimal_params})
zero_proj = (1 / 2**4 * (I + Z) ^ (I + Z) ^ (I + Z) ^ (I + Z))

zero_proj_meas = StateFn(zero_proj, is_measurement = True)
ansatz_and_opt_circ = ansatz.compose(opt_circ2.inverse())
#Penalty_ansatz = CircuitStateFn(ansatz_and_opt_circ)
Penalty_ansatz = CircuitStateFn(ansatz + opt_circ2.inverse())
Penalty_and_ansatz = zero_proj_meas @ Penalty_ansatz

H_tot2 = H_tot + 2 * Penalty_and_ansatz

def cost(params):
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    sampler = CircuitSampler(backend = backend)
    binded = H_tot2.bind_parameters({ry:params})
    sampled = sampler.convert(binded)
    return sampled.eval().real
params = np.random.random(num_params)
vqd2_result = minimize(cost, params, method="Powell", tol=1e-3)



In [52]:
print('The qeom excited state energy using is: {}'.format(qeom_results.eigenenergies[2]))
print('The estimated excited state energy from vqd algorithm is: {}'.format(vqd2_result.fun))


The qeom excited state energy using is: (-0.8999727657488661+0j)
The estimated excited state energy from vqd algorithm is: -1.2501376234722907


In [40]:
print(H_tot2)

SummedOp([
  ComposedOp([
    OperatorMeasurement(-0.8105479805373268 * IIII
    - 0.225753492224025 * ZIII
    + 0.17218393261915543 * IZII
    + 0.12091263261776641 * ZZII
    - 0.225753492224025 * IIZI
    + 0.17464343068300464 * ZIZI
    + 0.1661454325638243 * IZZI
    + 0.17218393261915546 * IIIZ
    + 0.1661454325638243 * ZIIZ
    + 0.16892753870087926 * IZIZ
    + 0.12091263261776641 * IIZZ
    + 0.04523279994605788 * XXXX
    + 0.04523279994605788 * YYXX
    + 0.04523279994605788 * XXYY
    + 0.04523279994605788 * YYYY),
    CircuitStateFn(
         »
    q_0: »
         »
    q_1: »
         »
    q_2: »
         »
    q_3: »
         »
    «     ┌──────────────────────────────────────────────────────────────────────────────┐
    «q_0: ┤0                                                                             ├
    «     │                                                                              │
    «q_1: ┤1                                                             